In [2]:
import pandas as pd
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS
from finrl.config import *
import itertools

In [3]:
TRAIN_START_DATE = '2023-05-01'
TRAIN_END_DATE = "2024-07-31"

TRADE_START_DATE = "2024-08-01"
TRADE_END_DATE = '2024-12-31'


TICKER_LIST = config_tickers.DOW_30_TICKER

dow30_df_finrl = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRADE_END_DATE,
                                ticker_list = TICKER_LIST).fetch_data()
    

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (12180, 8)


In [4]:
dow30_df_finrl.to_csv("prepared_csv/dow30.csv", index=False)

In [5]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(dow30_df_finrl)

processed
     


Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (419, 8)
Successfully added vix
Successfully added turbulence index


,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2023-05-01,167.324615,168.173128,166.387309,167.018759,52472900,AAPL,0,0.000000,168.271681,165.341616,0.000000,-66.666667,100.000000,167.324615,167.324615,16.08,0.000000
1,2023-05-01,218.973877,221.173707,218.231439,220.009631,1920500,AMGN,0,0.000000,168.271681,165.341616,0.000000,-66.666667,100.000000,218.973877,218.973877,16.08,0.000000
2,2023-05-01,154.589569,157.044449,153.968557,157.044449,1803600,AXP,0,0.000000,168.271681,165.341616,0.000000,-66.666667,100.000000,154.589569,154.589569,16.08,0.000000
3,2023-05-01,203.869995,208.520004,203.619995,206.250000,3776800,BA,0,0.000000,168.271681,165.341616,0.000000,-66.666667,100.000000,203.869995,203.869995,16.08,0.000000
4,2023-05-01,208.543396,213.045025,207.833113,210.530247,2876400,CAT,0,0.000000,168.271681,165.341616,0.000000,-66.666667,100.000000,208.543396,208.543396,16.08,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12146,2024-12-27,238.475906,241.271512,237.646119,238.189423,631700,TRV,4,-3.629097,264.034936,227.654548,46.563710,-84.103435,19.083843,249.527451,246.598195,15.95,5.254974
12147,2024-12-27,501.078979,504.213243,498.927243,500.518962,3471800,UNH,4,-21.282145,613.132531,443.280082,41.154831,-78.267738,33.447245,546.648778,557.935269,15.95,5.254974
12148,2024-12-27,316.412628,319.232595,315.320380,317.018312,3489200,V,4,3.688335,319.819458,304.959991,60.949982,138.598018,17.096441,311.156319,298.606561,15.95,5.254974
12149,2024-12-27,37.348583,37.573125,37.095978,37.292450,15168100,VZ,4,-0.668354,41.690306,36.140142,40.653697,-121.273541,30.592181,39.269344,39.397805,15.95,5.254974


In [6]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2023-05-01,AAPL,167.324615,168.173128,166.387309,167.018759,52472900.0,0.0,0.000000,168.271681,165.341616,0.000000,-66.666667,100.000000,167.324615,167.324615,16.08,0.000000
1,2023-05-01,AMGN,218.973877,221.173707,218.231439,220.009631,1920500.0,0.0,0.000000,168.271681,165.341616,0.000000,-66.666667,100.000000,218.973877,218.973877,16.08,0.000000
2,2023-05-01,AXP,154.589569,157.044449,153.968557,157.044449,1803600.0,0.0,0.000000,168.271681,165.341616,0.000000,-66.666667,100.000000,154.589569,154.589569,16.08,0.000000
3,2023-05-01,BA,203.869995,208.520004,203.619995,206.250000,3776800.0,0.0,0.000000,168.271681,165.341616,0.000000,-66.666667,100.000000,203.869995,203.869995,16.08,0.000000
4,2023-05-01,CAT,208.543396,213.045025,207.833113,210.530247,2876400.0,0.0,0.000000,168.271681,165.341616,0.000000,-66.666667,100.000000,208.543396,208.543396,16.08,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17598,2024-12-27,TRV,238.475906,241.271512,237.646119,238.189423,631700.0,4.0,-3.629097,264.034936,227.654548,46.563710,-84.103435,19.083843,249.527451,246.598195,15.95,5.254974
17599,2024-12-27,UNH,501.078979,504.213243,498.927243,500.518962,3471800.0,4.0,-21.282145,613.132531,443.280082,41.154831,-78.267738,33.447245,546.648778,557.935269,15.95,5.254974
17600,2024-12-27,V,316.412628,319.232595,315.320380,317.018312,3489200.0,4.0,3.688335,319.819458,304.959991,60.949982,138.598018,17.096441,311.156319,298.606561,15.95,5.254974
17601,2024-12-27,VZ,37.348583,37.573125,37.095978,37.292450,15168100.0,4.0,-0.668354,41.690306,36.140142,40.653697,-121.273541,30.592181,39.269344,39.397805,15.95,5.254974


In [7]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))
     


train.to_csv('prepared_csv/dow30_train_data.csv')
trade.to_csv('prepared_csv/dow30_trade_data.csv')

9106
3016
